In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
%matplotlib inline

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of cloudiness
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
clouds = city_data_df["Cloudiness"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=clouds)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
import gmaps
gmaps.configure(api_key='Your api key here')
new_york_coordinates = (40.75, -74.00)
gmaps.figure(center=new_york_coordinates, zoom_level=12)

In [8]:
# Heatmap of wind speed
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
wind = city_data_df["Wind Speed"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Vanimo,PG,2021-01-11 01:11:55,-2.6741,141.3028,84.58,70,100,2.95
1,1,Sao Filipe,CV,2021-01-11 01:06:54,14.8961,-24.4956,70.86,71,100,17.76
3,3,Erechim,BR,2021-01-11 01:20:17,-27.6342,-52.2739,78.80,78,20,4.61
6,6,Vila Franca Do Campo,PT,2021-01-11 01:11:40,37.7167,-25.4333,55.40,94,75,10.36
7,7,Comodoro Rivadavia,AR,2021-01-11 01:07:14,-45.8667,-67.5000,62.60,25,20,23.02
8,8,Denpasar,ID,2021-01-11 01:20:13,-8.6500,115.2167,82.40,83,40,2.30
10,10,Arraial Do Cabo,BR,2021-01-11 01:11:42,-22.9661,-42.0278,76.87,90,3,16.78
13,13,Rikitea,PF,2021-01-11 01:11:35,-23.1203,-134.9692,78.35,78,38,16.40
14,14,Hilo,US,2021-01-11 01:11:37,19.7297,-155.0900,82.40,54,1,10.36
18,18,Guane,CU,2021-01-11 01:20:18,22.2025,-84.0875,62.74,80,0,6.58


In [11]:
preferred_cities_df.count()

City_ID       293
City          293
Country       289
Date          293
Lat           293
Lng           293
Max Temp      293
Humidity      293
Cloudiness    293
Wind Speed    293
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vanimo,PG,84.58,-2.6741,141.3028,
1,Sao Filipe,CV,70.86,14.8961,-24.4956,
3,Erechim,BR,78.80,-27.6342,-52.2739,
6,Vila Franca Do Campo,PT,55.40,37.7167,-25.4333,
7,Comodoro Rivadavia,AR,62.60,-45.8667,-67.5000,
...,...,...,...,...,...,...
511,Puerto Del Rosario,ES,57.20,28.5004,-13.8627,
516,Praia Da Vitoria,PT,59.00,38.7333,-27.0667,
523,Buraydah,SA,51.80,26.3260,43.9750,
525,Portel,BR,75.74,-1.9356,-50.8211,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print ("Hotel not found ... skipping")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [15]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Vanimo,PG,84.58,-2.6741,141.3028,Vahmoneh Lodge Vanimo
1,Sao Filipe,CV,70.86,14.8961,-24.4956,Tortuga B&B
3,Erechim,BR,78.80,-27.6342,-52.2739,Hotel Buongiorno
6,Vila Franca Do Campo,PT,55.40,37.7167,-25.4333,Pestana Bahia Praia
7,Comodoro Rivadavia,AR,62.60,-45.8667,-67.5000,Austral Hotel
...,...,...,...,...,...,...
511,Puerto Del Rosario,ES,57.20,28.5004,-13.8627,Hotel JM Puerto del Rosario
516,Praia Da Vitoria,PT,59.00,38.7333,-27.0667,Hotel Branco I
523,Buraydah,SA,51.80,26.3260,43.9750,Mövenpick Hotel Qassim
525,Portel,BR,75.74,-1.9356,-50.8211,Kitnet do Marcos


In [16]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))